# Assignment 4
Jiaqi Guo    JGR9647

### Assignment Deep Q-Learning for Cartpole Balancing


In [1]:
import sys
from datetime import datetime
import random
import gym
import numpy as np
from collections import deque

import pylab
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.models import Model, load_model


# policy network
def OurModel(input_shape, action_space):
    X_input = Input(input_shape)

    # 'Dense' is the basic form of a neural network layer
    # Input Layer of state size(4) and Hidden Layer with 512 nodes
    X = Dense(512, input_shape=input_shape, activation="relu", kernel_initializer='he_uniform')(X_input)

    # Hidden layer with 256 nodes
    X = Dense(256, activation="relu", kernel_initializer='he_uniform')(X)

    # Hidden layer with 64 nodes
    X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X)

    # Output Layer with # of actions: 2 nodes (left, right)
    X = Dense(action_space, activation="linear", kernel_initializer='he_uniform')(X)

    model = Model(inputs=X_input, outputs=X, name='CartPole_DQN_model')
    model.compile(loss="mse", optimizer=RMSprop(lr=0.00025, rho=0.95, epsilon=0.01), metrics=["accuracy"])

    # model.summary()
    return model


class DQNAgent:
    def __init__(self):
        self.env = gym.make('CartPole-v1')
        # by default, CartPole-v1 has max episode steps = 500
        self.state_size = self.env.observation_space.shape[0]
        self.action_size = self.env.action_space.n
        self.EPISODES = 300
        self.memory = deque(maxlen=2000)

        self.gamma = 0.95  # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.001
        self.epsilon_decay = 0.999
        self.batch_size = 64
        self.train_start = 1000

        # create main model
        self.model = OurModel(input_shape=(self.state_size,), action_space=self.action_size)
        self.target_model = OurModel(input_shape=(self.state_size,), action_space=self.action_size)

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        if len(self.memory) > self.train_start:
            if self.epsilon > self.epsilon_min:
                self.epsilon *= self.epsilon_decay

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    # implement the epsilon-greedy policy
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        else:
            q_value = self.model.predict(state)
            return np.argmax(q_value[0])

    # implement the Q-learning
    def replay(self):
        if len(self.memory) < self.train_start:
            return
        # Randomly sample minibatch from the memory
        minibatch = random.sample(self.memory, min(len(self.memory), self.batch_size))

        state = np.zeros((self.batch_size, self.state_size))
        next_state = np.zeros((self.batch_size, self.state_size))
        action, reward, done = [], [], []

        # assign data into state, next_state, action, reward and done from minibatch
        for i in range(self.batch_size):
            state[i] = minibatch[i][0]
            action.append(minibatch[i][1])
            reward.append(minibatch[i][2])
            next_state[i] = minibatch[i][3]
            done.append(minibatch[i][4])

        # compute value function of current(call it target) and value function of next state(call it target_next)
        target = self.model.predict(state)
        target_val = self.target_model.predict(next_state)

        for i in range(self.batch_size):
            # correction on the Q value for the action used,
            # if done[i] is true, then the target should be just the final reward
            if done[i]:
                target[i][action[i]] = reward[i]
            else:
                target[i][action[i]] = reward[i] + self.gamma * (
                    np.amax(target_val[i]))

        # Train the Neural Network with batches where target is the value function
        self.model.fit(state, target, batch_size=self.batch_size,
                       epochs=1, verbose=0)

    def load(self, name):
        self.model = load_model(name)

    def save(self, name):
        self.model.save(name)

    def training(self):
        scores, episodes = [], []
        for e in range(self.EPISODES):
            state = self.env.reset()
            state = np.reshape(state, [1, self.state_size])
            done = False
            score = 0
            i = 0
            while not done:
                action = self.act(state)
                next_state, reward, done, _ = self.env.step(action)

                next_state = np.reshape(next_state, [1, self.state_size])
                if not done or i == 499:
                    reward = reward  # Reward --> +1
                else:
                    reward = -100  # Reward = -100

                self.remember(state, action, reward, next_state, done)
                self.replay()

                score += reward
                state = next_state

                i += 1
                if done:
                    dateTimeObj = datetime.now()
                    timestampStr = dateTimeObj.strftime("%H:%M:%S")

                    self.update_target_model()
                    score = score if score == 500 else score + 100

                    scores.append(score)
                    episodes.append(e)


                    print("episode: {}/{}, score: {}, e: {:.2}, time: {}".format(e + 1, self.EPISODES, i, self.epsilon,
                                                                                 timestampStr))
                    if np.mean(scores[-min(10, len(scores)):]) > 490:
                        return episodes, scores
                        # sys.exit()
        return episodes, scores


      # pylab.savefig("./save_graph/cartpole_dqn.png")

    # test function if you want to test the learned model
    def test(self):
        self.load("./save/cartpole-dqn-training.h5")
        for e in range(self.EPISODES):
            state = self.env.reset()
            state = np.reshape(state, [1, self.state_size])
            done = False
            i = 0
            while not done:
                # self.env.render()
                action = np.argmax(self.model.predict(state))
                next_state, reward, done, _ = self.env.step(action)
                state = np.reshape(next_state, [1, self.state_size])
                i += 1
                if done:
                    print("episode: {}/{}, score: {}".format(e + 1, self.EPISODES, i))
                    break

D:\anaconda3\envs\test_env\lib\site-packages\tensorflow\core\framework\resource_handle_pb2.py:23: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  serialized_pb=_b('\n/tensorflow/core/framework/resource_handle.proto\x12\ntensorflow\"r\n\x13ResourceHandleProto\x12\x0e\n\x06\x64\x65vice\x18\x01 \x01(\t\x12\x11\n\tcontainer\x18\x02 \x01(\t\x12\x0c\n\x04name\x18\x03 \x01(\t\x12\x11\n\thash_code\x18\x04 \x01(\x04\x12\x17\n\x0fmaybe_type_name\x18\x05 \x01(\tBn\n\x18org.tensorflow.frameworkB\x0eResourceHandleP\x01Z=github.com/tensorflow/tensorflow/tensorflow/go/core/framework\xf8\x01\x01\x62\x06proto3')
D:\anaconda3\envs\test_env\lib\site-packages\tensorflow\core\framework\resource_handle_pb2.py:42: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please

In [ ]:
agent = DQNAgent()
episodes, scores = agent.training()
# agent.test()

D:\anaconda3\envs\test_env\lib\site-packages\tensorflow\python\framework\tensor_util.py:538: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  tensor_proto.tensor_content = nparray.tostring()


episode: 1/300, score: 14, e: 1.0, time: 17:29:44
episode: 2/300, score: 15, e: 1.0, time: 17:29:44
episode: 3/300, score: 35, e: 1.0, time: 17:29:44
episode: 4/300, score: 26, e: 1.0, time: 17:29:44
episode: 5/300, score: 12, e: 1.0, time: 17:29:44
episode: 6/300, score: 15, e: 1.0, time: 17:29:44
episode: 7/300, score: 23, e: 1.0, time: 17:29:44
episode: 8/300, score: 19, e: 1.0, time: 17:29:44
episode: 9/300, score: 14, e: 1.0, time: 17:29:45
episode: 10/300, score: 34, e: 1.0, time: 17:29:45
episode: 11/300, score: 22, e: 1.0, time: 17:29:45
episode: 12/300, score: 39, e: 1.0, time: 17:29:45
episode: 13/300, score: 13, e: 1.0, time: 17:29:45
episode: 14/300, score: 11, e: 1.0, time: 17:29:45
episode: 15/300, score: 16, e: 1.0, time: 17:29:45
episode: 16/300, score: 85, e: 1.0, time: 17:29:45
episode: 17/300, score: 37, e: 1.0, time: 17:29:45
episode: 18/300, score: 15, e: 1.0, time: 17:29:45
episode: 19/300, score: 13, e: 1.0, time: 17:29:45
episode: 20/300, score: 23, e: 1.0, time

In [ ]:
pylab.plot(episodes, scores, 'b')
pylab.show()